In [184]:
import pandas as pd
import numpy as np
class bufer:
    def __init__(self) -> None:
        self.map = pd.DataFrame(columns=['destination_start', 'end', 'source_start', 'range'])

    def index_map(self, destination_start, source_start, range_):
        end = source_start + range_
        self.map.loc[len(self.map.index)] = destination_start, end, source_start, range_

    def get_next_id(self, id, debug=False):
        a = self.map[(self.map["source_start"]<=id) & (self.map["end"] >= id)]
        # print(a)
        diff = np.abs(id - a['source_start'])
        next_id = (a['destination_start'] + diff).values
        if (debug):
            print(a)
            print(diff)
        # print(next_id)
        return next_id[0] if len(next_id)>0 else id
    

    
    def get_next_id_by_set(self, start_id, start_size):
        new_set = []
        a = self.map[(self.map["source_start"]<=start_id)]
        setart_end = start_id + start_size
        if(a.shape[0] > 1):
            b = a.copy()
            c = a.apply(lambda df_a: abs(df_a["end"] - setart_end), axis=1)
            a.insert(0, "diff_end", list(c))
            a = a.sort_values(by="diff_end", ascending=True)
            a = a[(a["source_start"] <= start_id)& (a["end"] > start_id)].reset_index(drop=True)
            a = a.iloc[:1]
            # print("===================================================")
            # print(f"start_id-> {start_id} | start_size -> {start_size}")
            # print(a)
            # print("===================================================")
            
        if(a.shape[0] == 1):
            S = a["source_start"].values
            diff_startad = start_id - a["source_start"].values[0]
            if (a['end'].values[0] >= setart_end):
                new_destination_start = a['destination_start'].values[0] + diff_startad
                new_range = start_size
                new_set.append((new_destination_start, new_range))
                # print(f"A-> {start_id, start_size, S, new_set}")

                # print('aaa')
                # print(f"Diff -> {diff_startad}")

            else:
                new_start = a['end'].values[0]
                new_range = (start_id + start_size) - a['end'].values[0]
                new_set.append((a['destination_start'].values[0] + diff_startad, a['end'].values[0] - start_id))
                sub_set = self.get_next_id_by_set(new_start, new_range)
                new_set += sub_set
                # print(f"B-> {start_id, start_size, S, new_set}")

        else:
            new_set.append((start_id, start_size))
            # print(f"C-> {start_id, start_size, new_set}")


        # print("===================================================")
        # print(f"start_id-> {start_id} | start_size -> {start_size}")
        # print(new_set)
        # print("===================================================")
        
        return new_set

In [193]:
import json
f = open('/data/repor/adventofcode/5/data_1.json')
data = json.load(f)
f.close


<function TextIOWrapper.close()>

In [194]:
names = list(data.keys())
names.pop(0)
start = data.get('start')

all_bufers = {}

for name in names:
    bufer_data = data.get(name, [])
    if bufer_data == []:
        raise IndexError("No data")
    bufer_aux = bufer()
    for destination_start, source_start, range_ in bufer_data:
        bufer_aux.index_map(destination_start, source_start, range_)
    all_bufers.update({name: bufer_aux})

In [195]:
end_ids = []
for seed in start:
    id = seed
    for name in names:
        id = all_bufers.get(name).get_next_id(id)
    end_ids.append(id)
    print(f"Start -> {seed} | location -> {id}")
min_id = min(end_ids)
print(min_id)

Start -> 929142010 | location -> 2928341086
Start -> 467769747 | location -> 2743737607
Start -> 2497466808 | location -> 2306805090
Start -> 210166838 | location -> 1471269794
Start -> 3768123711 | location -> 1787915678
Start -> 33216796 | location -> 4013772529
Start -> 1609270159 | location -> 3872583593
Start -> 86969850 | location -> 2760787031
Start -> 199555506 | location -> 2768526595
Start -> 378609832 | location -> 2290868489
Start -> 1840685500 | location -> 3135505386
Start -> 314009711 | location -> 2418319456
Start -> 1740069852 | location -> 2064075662
Start -> 36868255 | location -> 4017423988
Start -> 2161129344 | location -> 3651186410
Start -> 170490105 | location -> 3322203336
Start -> 2869967743 | location -> 1377440429
Start -> 265455365 | location -> 1249872241
Start -> 3984276455 | location -> 600279879
Start -> 31190888 | location -> 4011746621
600279879


In [197]:
#  2
import cProfile

def example_2():
    newStart = np.array_split(start, len(start)/2)
    newStart = sorted(newStart, key= lambda x: x[0])
    newStart = [tuple(i.tolist()) for i in newStart]

    end_ids = []
    for seed in newStart:
        id = [seed]
        for name in names:
            sub_set = []
            for start_id, start_size in id:
                sub_set += all_bufers.get(name).get_next_id_by_set(start_id, start_size)
            id = sub_set
        end_ids += id
        # print(f"Start -> {seed} | location -> {id}")
    min_id = min(end_ids, key=lambda x: x[0])
    print(min_id)

cProfile.run('example_2()')

(20191102, 3655255)
         1387359 function calls (1357916 primitive calls) in 1.874 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
       95    0.000    0.000    0.000    0.000 2489403604.py:19(<lambda>)
        1    0.008    0.008    1.874    1.874 2489403604.py:4(example_2)
       10    0.000    0.000    0.000    0.000 2489403604.py:6(<lambda>)
        1    0.000    0.000    0.000    0.000 2489403604.py:7(<listcomp>)
  433/348    0.036    0.000    1.865    0.005 3948026618.py:22(get_next_id_by_set)
     6354    0.010    0.000    0.087    0.000 3948026618.py:28(<lambda>)
     2490    0.001    0.000    0.004    0.000 <frozen abc>:117(__instancecheck__)
      415    0.001    0.000    0.001    0.000 <frozen importlib._bootstrap>:1207(_handle_fromlist)
        1    0.000    0.000    1.874    1.874 <string>:1(<module>)
    11512    0.005    0.000    0.005    0.000 __init__.py:33(using_copy_on_write)
      415    0.000    0.

In [192]:
# test
seed = bufer()
seed.index_map(50,98,2)
seed.index_map(52,50,48)

soil = bufer()
soil.index_map(0, 15, 37)
soil.index_map(37, 52, 2)
soil.index_map(39, 0, 15)

fertilizer = bufer()
fertilizer.index_map(49, 53, 8)
fertilizer.index_map(0, 11, 42)
fertilizer.index_map(42, 0, 7)
fertilizer.index_map(57, 7, 4)

water = bufer()
water.index_map(88, 18, 7)
water.index_map(18, 25, 70)

light = bufer()
light.index_map(45, 77, 23)
light.index_map(81, 45, 19)
light.index_map(68, 64, 13)

temperature = bufer()
temperature.index_map(0, 69, 1)
temperature.index_map(1, 0, 69)

humidity= bufer()
humidity.index_map(60, 56, 37)
humidity.index_map(56, 93, 4)

id= [(55, 15)]
print(f"seed: {id}")

sub_set = []
for start_id, start_size in id:
    sub_set += seed.get_next_id_by_set(start_id, start_size)
id = sub_set
print(f"soil: {id}")

sub_set = []
for start_id, start_size in id:
    sub_set += soil.get_next_id_by_set(start_id, start_size)
id = sub_set
print(f"fertilizer: {id}")


sub_set = []
for start_id, start_size in id:
    sub_set += fertilizer.get_next_id_by_set(start_id, start_size)
id = sub_set
print(f"water: {id}")

sub_set = []
for start_id, start_size in id:
    sub_set += water.get_next_id_by_set(start_id, start_size)
id = sub_set
print(f"light: {id}")

sub_set = []
for start_id, start_size in id:
    sub_set += light.get_next_id_by_set(start_id, start_size)
id = sub_set
print(f"temperature: {id}")

sub_set = []
for start_id, start_size in id:
    sub_set += temperature.get_next_id_by_set(start_id, start_size)
id = sub_set
print(f"humidity: {id}")

sub_set = []
for start_id, start_size in id:
    sub_set += humidity.get_next_id_by_set(start_id, start_size)
id = sub_set
print(f"Final: {id}")

seed: [(55, 15)]
soil: [(57, 15)]
fertilizer: [(57, 15)]
water: [(53, 4), (61, 11)]
light: [(46, 4), (54, 11)]
temperature: [(82, 4), (90, 10), (68, 1)]
humidity: [(82, 4), (90, 10), (69, 1)]
Final: [(86, 4), (94, 3), (56, 4), (97, 3), (73, 1)]


In [206]:
# Test

seed = bufer()
seed.index_map(50,98,2)
seed.index_map(52,50,48)

soil = bufer()
soil.index_map(0, 15, 37)
soil.index_map(37, 52, 2)
soil.index_map(39, 0, 15)

fertilizer = bufer()
fertilizer.index_map(49, 53, 8)
fertilizer.index_map(0, 11, 42)
fertilizer.index_map(42, 0, 7)
fertilizer.index_map(57, 7, 4)

water = bufer()
water.index_map(88, 18, 7)
water.index_map(18, 25, 70)

light = bufer()
light.index_map(45, 77, 23)
light.index_map(81, 45, 19)
light.index_map(68, 64, 13)

temperature = bufer()
temperature.index_map(0, 69, 1)
temperature.index_map(1, 0, 69)

humidity= bufer()
humidity.index_map(60, 56, 37)
humidity.index_map(56, 93, 4)


id=79
print(f"seed: {id}")

id = seed.get_next_id(id)
print(f"soil: {id}")

id = soil.get_next_id(id)
print(f"fertilizer: {id}")

id = fertilizer.get_next_id(id)
print(f"water: {id}")

id = water.get_next_id(id)
print(f"light: {id}")

id = light.get_next_id(id)
print(f"temperature: {id}")

id = temperature.get_next_id(id)
print(f"humidity: {id}")

id = humidity.get_next_id(id)
print(f"Final: {id}")


seed: 79
soil: 81
fertilizer: 81
water: 81
light: 74
temperature: 78
humidity: 78
Final: 82
